In [12]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import AzureOpenAI
import os
import openai
from dotenv import load_dotenv

In [13]:
import pandas as pd

df = pd.read_csv('./mydata/titanic.csv')

In [14]:

load_dotenv()
# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
# Create a completion - qna를 위하여 davinci 모델생성
llm = AzureOpenAI(deployment_name="text-davinci-003")

USING API_BASE: 
https://oai-sqlgpt.openai.azure.com/


In [17]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

In [18]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to check the size of the dataframe
Action: python_repl_ast
Action Input: df.shape
Observation: (891, 12)
Thought: I can see the number of rows is 891
Final Answer: There are 891 rows.

> Finished chain.


'There are 891 rows.'

In [19]:
agent.run("show me each count group by gender?")



> Entering new AgentExecutor chain...
Thought: I need to group by gender and then count
Action: python_repl_ast
Action Input: df.groupby('Sex').count()
Observation:         PassengerId  Survived  Pclass  Name  Age  SibSp  Parch  Ticket  Fare  \
Sex                                                                            
female          314       314     314   314  261    314    314     314   314   
male            577       577     577   577  453    577    577     577   577   

        Cabin  Embarked  
Sex                      
female     97       312  
male      107       577  
Thought: I now know the final answer
Final Answer: for females, there are 314 passengers, for males, there are 577 passengers.

> Finished chain.


'for females, there are 314 passengers, for males, there are 577 passengers.'

In [20]:
agent.run("전체 컬럼 이름들을 알려줘")



> Entering new AgentExecutor chain...
Thought: I need to get a list of the column names
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Thought: I now know the final answer
Final Answer: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

> Finished chain.


"['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']"

In [23]:
agent.run("4명 이상의 형제를 가진사람은 몇명이야?")



> Entering new AgentExecutor chain...
Thought: 나는 형제의 수가 4보다 큰 값을 갖는 행을 찾아야한다.
Action: python_repl_ast
Action Input: df[df['SibSp'] >= 4]
Observation:      PassengerId  Survived  Pclass                                     Name  \
16            17         0       3                     Rice, Master. Eugene   
50            51         0       3               Panula, Master. Juha Niilo   
59            60         0       3       Goodwin, Master. William Frederick   
68            69         1       3          Andersson, Miss. Erna Alexandra   
71            72         0       3               Goodwin, Miss. Lillian Amy   
119          120         0       3        Andersson, Miss. Ellis Anna Maria   
159          160         0       3               Sage, Master. Thomas Henry   
164          165         0       3             Panula, Master. Eino Viljami   
171          172         0       3                     Rice, Master. Arthur   
180          181         0       3             Sage, Miss

'4명 이상의 형제를 가진사람은 30명이다.'

In [22]:
agent.run("how many people have more than 3 sibligngs")



> Entering new AgentExecutor chain...
Thought: I must find out how many passengers have more than 3 siblings
Action: python_repl_ast
Action Input: df['SibSp'].value_counts() > 3
Observation: 0    True
1    True
2    True
4    True
3    True
8    True
5    True
Name: SibSp, dtype: bool
Thought: I need to count how many True results there are
Action: python_repl_ast
Action Input: df['SibSp'].value_counts() > 3).sum()
Observation: unmatched ')' (<unknown>, line 1)
Thought: I need to add parenthesis to the input
Action: python_repl_ast
Action Input: (df['SibSp'].value_counts() > 3).sum()
Observation: 7
Thought: I now know the final answer
Final Answer: 7 passengers have more than 3 siblings.

> Finished chain.


'7 passengers have more than 3 siblings.'

In [24]:
agent.run("성별로 구분해서 각각 몇명이 살아있어?")



> Entering new AgentExecutor chain...
Thought: 데이터 프레임의 성별 열을 이용해서 각각 몇명이 살아있는지 확인해야한다.
Action: python_repl_ast
Action Input: df.groupby('Sex').Survived.sum()
Observation: Sex
female    233
male      109
Name: Survived, dtype: int64
Thought: 이렇게 나온 결과로 여성과 남성이 각각 몇 명 살아있는지 확인할 수 있다.
Final Answer: 여성은 233명, 남성은 109명이 살아있었습니다.

> Finished chain.


'여성은 233명, 남성은 109명이 살아있었습니다.'

In [25]:
agent.run("전체 데이터를 분석해보면 어떤 사람이 살아 있을 확률이 클까?")



> Entering new AgentExecutor chain...
Thought: 나이, 성별, 티켓 클래스에 따른 살아남은 확률을 분석해보자.
Action: python_repl_ast
Action Input: df.groupby(['Sex', 'Age', 'Pclass'])['Survived'].mean()
Observation: Sex     Age    Pclass
female  0.75   3         1.00
        1.00   3         1.00
        2.00   1         0.00
               2         1.00
               3         0.25
                         ... 
male    70.00  2         0.00
        70.50  3         0.00
        71.00  1         0.00
        74.00  3         0.00
        80.00  1         1.00
Name: Survived, Length: 283, dtype: float64
Thought: 이렇게 분석한 데이터를 바탕으로 살아남은 확률을 상위 3개 성별, 나이, 티켓 클래스를 기준으로 나열해보자.
Action: python_repl_ast
Action Input: df.groupby(['Sex', 'Age', 'Pclass'])['Survived'].mean().nlargest(3)
Observation: Sex     Age   Pclass
female  0.75  3         1.0
        1.00  3         1.0
        2.00  2         1.0
Name: Survived, dtype: float64
Thought: 이렇게 분석한 데이터를 바탕으로 살아남은 확률이 가장 높은 사람은 누구인가?
Action: python_repl_ast
Action Input

'살아남은 확률이 가장 높은 사람들의 이름은 Baclini, Miss. Helene Barbara과 Baclini, Miss. Eugenie 입니다.'

In [26]:
agent.run("전체 데이터를 분석해보면 살아 있을 확률이 가장높은 상위 3개의 성별, 나이, 티켓 클래스를 알려줘?")



> Entering new AgentExecutor chain...
Thought: 데이터를 분석해야 함
Action: python_repl_ast
Action Input: df.groupby(['Sex', 'Age', 'Pclass'])['Survived'].mean().sort_values(ascending=False).head(3)
Observation: Sex     Age    Pclass
female  0.75   3         1.0
        44.00  1         1.0
        40.00  2         1.0
Name: Survived, dtype: float64
Thought: 3개의 상위 확률을 알게 되었음
Final Answer: 살아 있을 확률이 가장 높은 상위 3개의 성별, 나이, 티켓 클래스는 각각 여성, 0.75 세, 티켓 클래스 1과 여성, 44세, 티켓 클래스 1, 여성, 40세, 티켓 클래스 2 입니다.

> Finished chain.


'살아 있을 확률이 가장 높은 상위 3개의 성별, 나이, 티켓 클래스는 각각 여성, 0.75 세, 티켓 클래스 1과 여성, 44세, 티켓 클래스 1, 여성, 40세, 티켓 클래스 2 입니다.'

In [29]:
agent.run("전체 데이터를 분석해보면, 남성 40대 티켓클래스 1 라고하면 살아 있을 확률이 얼마나될까?")



> Entering new AgentExecutor chain...
Thought: 남성 40대 티켓클래스 1 인 행들만 찾아서 그 중에서 살아남은 행을 찾아야 한다.
Action: python_repl_ast
Action Input: df[(df['Sex']=='male') & (df['Age']>=40) & (df['Pclass']==1)]
Observation:      PassengerId  Survived  Pclass  \
6              7         0       1   
30            31         0       1   
35            36         0       1   
54            55         0       1   
62            63         0       1   
92            93         0       1   
96            97         0       1   
110          111         0       1   
124          125         0       1   
155          156         0       1   
170          171         0       1   
174          175         0       1   
187          188         1       1   
209          210         1       1   
245          246         0       1   
252          253         0       1   
262          263         0       1   
263          264         0       1   
331          332         0       1   
339          340         0     

'남성 40대 티켓클래스 1 라고하면 살아 있을 확률은 28.3% 이다.'